In [0]:
import sys

sys.path.append("../include_utils/")

import ipyparallel as ipp
import os, time
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
import dill
from scipy import stats
from IPython.display import display
import geopy

samtools = "/home/cfriedline/gpfs/src/samtools-1.3/samtools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
picard = "/home/cfriedline/gpfs/src/broadinstitute-picard-03a1d72/dist/picard.jar"
java = "/home/cfriedline/g/src/jdk1.8.0_60/bin/java"
perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"

vcfutils = "perl /home/cfriedline/g/src/bcftools-1.3/vcfutils.pl"
vcftools = "/home/cfriedline/bin/vcftools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
tabix = "/home/cfriedline/gpfs/src/htslib-1.3/tabix"
bgzip = "/home/cfriedline/gpfs/src/htslib-1.3/bgzip"


def setup_r():
    os.environ['R_HOME'] = '/home/cfriedline/g/R3/lib64/R'
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'],
                                                     "/home/cfriedline/lib64")

setup_r()
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
r = robjects.r

%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext rpy2.ipython

In [0]:
ni_dir = "/home/cfriedline/eckertlab/Mitra/dDocent"
#imp_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40"
notimputed_vcf_gz = os.path.join(ni_dir, "isect_snps.recode.vcf.gz_sorted.vcf.gz")
#imputed_vcf_gz = os.path.join(imp_dir, "isect_snps.recode.vcf.gz_sorted.vcf.gz")
vcf_files = [notimputed_vcf_gz]#, imputed_vcf_gz]

In [0]:
hierf_trans = {0:11, 1:12, 2:22, -1:'NA'}
def apply_hierf_trans(series):
    return [hierf_trans[x] if x in hierf_trans else x for x in series]

In [0]:
def read_df(dirname, fname):
    f = os.path.join(dirname, "%s.txt" % fname)
    return pd.read_csv(f, sep="\t", index_col=0)

In [0]:
z12_swapped = [read_df(x, "z12_swapped") for x in [ni_dir]]

In [0]:
z12_swapped[0].head()

In [0]:
pops = pd.read_csv("~/eckertlab/Mitra/Pops.txt", sep="\t")

In [0]:
pops = pops.drop_duplicates()


In [0]:
pops.head()

In [0]:
z12_swapped[0].head()

In [0]:
def get_species_label(row):
    return pops[pops.Pop==row.population].Species.values[0]

z12_swapped[0]['species'] = z12_swapped[0].apply(get_species_label, axis=1)

In [0]:
z12_swapped[0]

In [0]:
hierf_df = [x.ix[:,:-3].apply(apply_hierf_trans) for x in z12_swapped]

In [0]:
hierf_df[0].shape

In [0]:
for i, df in enumerate(hierf_df):
    hierf_df[i] = z12_swapped[i][['species', 'population']].join(df)

In [0]:
hierf_df[0].head()

In [0]:
for i, elem in enumerate(vcf_files):
    filedir = os.path.dirname(elem)
    outfile = os.path.join(filedir, "isect_hierfstat.txt")
    hierf_df[i].to_csv(outfile, header=True, index=False, sep="\t")

# Put into R (because it can be slow)

```
library(hierfstat)
library(data.table)
data = fread("isect_hierfstat.txt", header=T, sep="\t", data.table=F)
data$population = as.factor(data$population)
data$species = as.factor(data$species)
data2 = data[,c(1, 3:ncol(data))]
levels1 = data.frame(data[,1:2])
levels2 = data.frame(data[,2])
loci = data[,3:ncol(data)]
bs = basic.stats(data2)
saveRDS(bs, "isect_hierfstat_basic_stats.rds")
res1 = varcomp.glob(levels=levels1, loci=loci, diploid=T)
res2 = varcomp.glob(levels=levels2, loci=loci, diploid=T)
saveRDS(res1, "isect_hierfstat_varcomp_nested_in_spp.rds")
saveRDS(res2, "isect_hierfstat_varcomp.rds")
```

### Also compute pairwise Fst (qrsh)
```
rm(list=ls())
library(hierfstat)
library(data.table)
library(snow)
data = fread("isect_hierfstat.txt", header=T, sep="\t", data.table=F)
levels = data.frame(data$popid)
loci = data[,2:ncol(data)]

run_varcomp = function(idx) {
    i = args[[idx]]$i
    j = args[[idx]]$j
    key = paste(i, j, sep="-")
    d = copy(data)
    d = subset(d, d$popid == i | d$popid == j)
    levels = data.frame(d$popid)
    loci = d[,2:ncol(d)]
    return(varcomp.glob(levels=levels, loci=loci, diploid=T))
}

args = list()
for (i in 1:6) {
    for (j in 1:i) {
        if (i != j) {
            args[[length(args)+1]] = list(i=i, j=j)
        }
    }
}

hosts = rep("localhost", length(args))
cl = makeSOCKcluster(hosts)
clusterExport(cl, "data")
clusterEvalQ(cl, library(hierfstat))
clusterEvalQ(cl, library(data.table))
clusterExport(cl, "args")
clusterExport(cl, "run_varcomp")
pairwise_res = parLapply(cl, 1:length(args), "run_varcomp")
saveRDS(pairwise_res, "isect_hierfstat_pairwise.rds")
saveRDS(args, "isect_hierfstat_pairwise_args.rds")
stopCluster(cl)

```

In [0]:
%%R
varcomp_not = readRDS("/home/cfriedline/eckertlab/Mitra/dDocent/isect_hierfstat_varcomp.rds")
varcomp_nested = readRDS("/home/cfriedline/eckertlab/Mitra/dDocent/isect_hierfstat_varcomp_nested_in_spp.rds")
bs_not = readRDS("/home/cfriedline/eckertlab/Mitra/dDocent/isect_hierfstat_basic_stats.rds")

In [0]:
def get_r_series(key):
    s = pd.Series(get_r(key))
    s.index = get_r("names(%s)" % key)
    return s

def get_r_df(key):
    df = pd.DataFrame(get_r(key))
    try:
        rname = get_r("rownames(%s)" % key)
        df.index = rname
    except:
        pass
    
    try:
        cname = get_r("colnames(%s)" % key)
        df.columns = cname
    except:
        pass
    
    return df

def get_r(key):
    return r(key)

In [0]:
perloc = get_r_df("bs_not$perloc")
Ho = get_r_df("bs_not$Ho")
Hs = get_r_df("bs_not$Hs")
Fis = get_r_df("bs_not$Fis")
overall = get_r_series("bs_not$overall")
n_ind_samp = get_r_df("bs_not$n.ind.samp")


In [0]:
loc_df_pop = get_r_df('varcomp_not$loc')
F_df_pop = get_r_df('varcomp_not$F')
overall_df_pop = get_r_df('varcomp_not$overall')

loc_df_spp = get_r_df('varcomp_nested$loc')
F_df_spp = get_r_df('varcomp_nested$F')
overall_df_spp = get_r_df('varcomp_nested$overall')

In [0]:
F_df_pop

In [0]:
F_df_spp

In [0]:
def compute_fst(series):
    Hs = series[0]
    Ht = sum(series)
    return Hs/Ht

def compute_nested_fst(series):
    Hs = sum(series[0:2])
    Ht = sum(series)
    return Hs/Ht

In [0]:
loci_fst_pop = loc_df_pop.apply(compute_fst, axis=1)
loci_fst_spp = loc_df_spp.apply(compute_fst, axis=1)
loci_fst_spp_pop = loc_df_spp.apply(compute_nested_fst, axis=1)


In [0]:
fst_df = pd.DataFrame(loci_fst_pop).join(pd.DataFrame(loci_fst_spp), lsuffix="_pop", rsuffix="_spp")
fst_df.columns = ["(pop, total)", "(spp, total)"]

In [0]:
loci_fst_spp_pop.name = "(spp+pop, total)"
fst_df = fst_df.join(pd.DataFrame(loci_fst_spp_pop))

In [0]:
fst_df["spp_sums"] = fst_df.iloc[:,0] + fst_df.iloc[:,1]

In [0]:
fst_df[fst_df < 0] = 0

In [0]:
fst_df.head()

In [0]:
fst_cutoff = fst_df["(spp+pop, total)"].quantile(.75)

In [0]:
fst_cutoff

In [0]:
high_fst_snps = pd.DataFrame(fst_df[fst_df["(spp+pop, total)"] > fst_cutoff])

In [0]:
len(high_fst_snps)*4

In [0]:
sns.distplot(high_fst_snps["(spp+pop, total)"], kde=False, color="orange")
sns.distplot(fst_df["(spp+pop, total)"], kde=False)
plt.title("top 25% of SNPs(Fst > {}, n = {})".format(np.round(fst_cutoff, 4), len(high_fst_snps)))
plt.ylim(0, 5000)
plt.show()

In [0]:
sns.set_context("talk")
sns.set_style("white")
sns.regplot("(pop, total)", "(spp, total)", data=fst_df)
plt.show()

In [0]:
res = stats.linregress(fst_df.iloc[:,0], fst_df.iloc[:,1])
res

In [0]:
popid_map = {}
for population, data in z12_swapped[0].groupby("population"):
    popid = data['popid'].unique()[0]
    print(population, popid)
    popid_map[popid] = population

In [0]:
high_fst_snps["contig"] = high_fst_snps.apply(lambda x: x.name.split("-")[0], axis=1)
high_fst_snps["position"] = high_fst_snps.apply(lambda x: int(x.name.split("-")[1]), axis=1)


In [0]:
high_fst_snps[['contig', 'position', "(spp+pop, total)"]].to_csv(os.path.join(ni_dir, "high_fst_snps.txt"), sep="\t", header=True, index=None)

In [0]:
from pyfasta import Fasta

In [0]:
assembly = os.path.join(ni_dir, "reference.fasta")

In [0]:
ref = Fasta(assembly)

In [0]:
with open(os.path.join(ni_dir, "high_fst_snps.fasta"), "w") as o:
    for c in high_fst_snps.contig.values:
        o.write(">{}\n{}\n".format(c, ref[c]))